<a href="https://colab.research.google.com/github/Ramsiit2010/OCIGenAIBot/blob/main/RCOE_OCIGenAIBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Run Your GitHub Flask App on Google Colab (with Public URL)

**This notebook will:**
1) Clone your GitHub repo
2) Install Python requirements
3) Run your Flask app via Gunicorn on port 5000
4) Expose a public HTTPS URL using ngrok

➡️ Just work top-to-bottom. Fill in your repo/config in the next cell.


In [14]:
REPO_URL = "https://github.com/Ramsiit2010/OCIGenAIBot.git"  # <-- put your repo URL
PROJECT_DIR_NAME = "OCIGenAIBot"  # folder name after clone (edit if different)

# If your app exposes a Flask instance named `app`, set it here as module:variable
# Examples:
#   Simple app at repo root in app.py:        APP_MODULE = "app:app"
#   Package layout myapp/__init__.py -> app:  APP_MODULE = "myapp:app"
#   run.py that creates app variable:         APP_MODULE = "run:app"
APP_MODULE = "AskMeChatBot:app"  # <-- change to match your entry point

# Optional: If your repo needs environment variables, set them here
EXTRA_ENVS = {
    # "FLASK_ENV": "development",
    # "SOME_API_KEY": "",
}

# 🔐 (Optional but recommended) Your ngrok Authtoken for better reliability
# Get one free at https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTHTOKEN = "35KIlIDMzheum8MLvD4kqv39A4q_RD1X1u3CM4MhztuXkadu"  # paste token here or leave blank

In [2]:
%%bash
python -V
pip install --upgrade pip >/dev/null
pip install flask gunicorn pyngrok >/dev/null
echo "✅ Installed: flask, gunicorn, pyngrok"

Python 3.12.12
✅ Installed: flask, gunicorn, pyngrok


In [5]:
import os, shutil, subprocess, sys

if os.path.exists(PROJECT_DIR_NAME):
    shutil.rmtree(PROJECT_DIR_NAME)

print("Cloning repo...", REPO_URL)
rc = subprocess.call([sys.executable, "-m", "pip", "install", "gitpython"], stdout=subprocess.DEVNULL)
from git import Repo
Repo.clone_from(REPO_URL, PROJECT_DIR_NAME)
print("✅ Repo cloned into:", PROJECT_DIR_NAME)


Cloning repo... https://github.com/Ramsiit2010/OCIGenAIBot.git
✅ Repo cloned into: OCIGenAIBot


In [6]:
import os, subprocess, sys
req_path = os.path.join(PROJECT_DIR_NAME, "requirements.txt")
if os.path.exists(req_path):
    print("Installing requirements.txt...")
    rc = subprocess.call([sys.executable, "-m", "pip", "install", "-r", req_path])
    if rc != 0:
        raise RuntimeError("pip install -r requirements.txt failed")
    print("✅ Requirements installed")
else:
    print("⚠️ No requirements.txt found — skipping")


Installing requirements.txt...
✅ Requirements installed


In [7]:
import os
for k, v in EXTRA_ENVS.items():
    os.environ[k] = str(v)
print("Set envs:", list(EXTRA_ENVS.keys()))


Set envs: []


In [10]:
import os, importlib, sys
sys.path.insert(0, PROJECT_DIR_NAME)

module_name, var_name = APP_MODULE.split(":", 1)
print(f"🔎 Importing {module_name}.{var_name} ...")
mod = importlib.import_module(module_name)
assert hasattr(mod, var_name), f"Could not find '{var_name}' in module '{module_name}'. Update APP_MODULE."
print("✅ Flask app reference found.")


🔎 Importing AskMeChatBot.app ...


✅ Flask app reference found.


In [11]:
import os, subprocess, signal, time
from pathlib import Path

PORT = 5000
LOGFILE = "/tmp/gunicorn.log"

if os.path.exists(LOGFILE):
    os.remove(LOGFILE)

cmd = [
    "gunicorn",
    APP_MODULE,
    "--bind", f"0.0.0.0:{PORT}",
    "--workers", "1",
    "--threads", "4",
    "--timeout", "120",
]

print("Starting Gunicorn:", " ".join(cmd))
gunicorn_proc = subprocess.Popen(cmd, cwd=PROJECT_DIR_NAME, stdout=open(LOGFILE, "a"), stderr=subprocess.STDOUT)
time.sleep(3)
print("✅ Gunicorn started (PID)", gunicorn_proc.pid)
print("📄 Tail logs with the next cell if needed.")


Starting Gunicorn: gunicorn AskMeChatBot:app --bind 0.0.0.0:5000 --workers 1 --threads 4 --timeout 120
✅ Gunicorn started (PID) 2368
📄 Tail logs with the next cell if needed.


In [12]:
import time
LOGFILE = "/tmp/gunicorn.log"
print("--- Gunicorn Logs (last 200 lines) ---")
try:
    with open(LOGFILE, 'r') as f:
        lines = f.readlines()[-200:]
        for line in lines:
            print(line, end='')
except FileNotFoundError:
    print("No logs yet.")


--- Gunicorn Logs (last 200 lines) ---
[2025-11-11 09:12:23 +0000] [2368] [INFO] Starting gunicorn 23.0.0
[2025-11-11 09:12:23 +0000] [2368] [INFO] Listening at: http://0.0.0.0:5000 (2368)
[2025-11-11 09:12:23 +0000] [2368] [INFO] Using worker: gthread
[2025-11-11 09:12:23 +0000] [2373] [INFO] Booting worker with pid: 2373
OCI SDK not available: cannot import name 'ChatDetails' from 'oci.generative_ai_inference.models' (/usr/local/lib/python3.12/dist-packages/oci/generative_ai_inference/models/__init__.py). Install with: pip install oci
2025-11-11 09:12:23,952 - INFO - Application initialized
2025-11-11 09:12:23,952 - INFO - Configuration loaded successfully from /content/OCIGenAIBot/config.properties
2025-11-11 09:12:23,952 - INFO - Loaded API spec: api_spec_general.json
2025-11-11 09:12:23,952 - INFO - Loaded API spec: api_spec_finance.json
2025-11-11 09:12:23,952 - INFO - Loaded API spec: api_spec_hr.json
2025-11-11 09:12:23,953 - INFO - Loaded API spec: api_spec_orders.json
2025-11

In [15]:
from pyngrok import ngrok, conf
import time

if NGROK_AUTHTOKEN:
    conf.get_default().auth_token = NGROK_AUTHTOKEN
    print("🔐 ngrok authtoken set")
else:
    print("ℹ️ No ngrok authtoken set — using anonymous session")

print("Starting ngrok tunnel to 5000...")
public_tunnel = ngrok.connect(5000, proto="http")
public_url = public_tunnel.public_url
print("🌍 Public URL:", public_url)
print("Open this in your browser. Keep this notebook running.")


🔐 ngrok authtoken set
Starting ngrok tunnel to 5000...
🌍 Public URL: https://irremediable-unbenumbed-ellison.ngrok-free.dev
Open this in your browser. Keep this notebook running.


## ✅ You're Live!

- Your app is running on port **5000** inside Colab.
- ngrok is exposing a **Public URL** printed above.
- **Keep this notebook open**; if the runtime sleeps, the URL will stop.

### Troubleshooting
- If you see `ModuleNotFoundError`, make sure `requirements.txt` includes all libs and re-run the *Install requirements* cell.
- If Gunicorn can't import your app, double-check `APP_MODULE` in the config cell.
- Use the **Tail logs** cell to see server errors.
